In [1]:
import numpy as np
from scipy import integrate
import math
import numpy as np, matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error as MAE
import time
import tqdm
from tqdm import trange
from environment import *

In [2]:
import pandas as pd
Liu_all=pd.read_csv('Liu_all.csv')

In [3]:
# Liu_all['ar_ratio']=np.random.uniform(1.3,1.8,len(Liu_all))
# Liu_all.to_csv('Liu_all.csv')

In [4]:
env= environment()
jc_scale=6
env.update_fitted_para(0.01,0.03)

In [5]:
tcd8_target=Liu_all['tcd8_pop']
r_l=np.ones(len(Liu_all))*0.1

In [8]:

def opt_fix_para():
    time_l=[]
    cd8_l=[]
    for i in trange(len(Liu_all)):
        mu=Liu_all.loc[i,'mu']
        r=r_l[i]
        jc=Liu_all.loc[i,'mhc_norm']
        jc_=(1-np.exp(-jc*jc_scale))/(1+np.exp(-jc*jc_scale))
        R_L_prop=Liu_all.loc[i,'Treg_pro']
        tumor=Liu_all.loc[i,'tumor_size']
        a=Liu_all.loc[i,'a']
        ar_ratio=Liu_all.loc[i,'ar_ratio']
        t1=Liu_all.loc[i,'time']
        
        x_=env.run(mu,r,jc_,R_L_prop,t1,a,ar_ratio)
        j=cal_time(x_,tumor)
        cd8_pred=min(x_[j][2]/(x_[j][0]+x_[j][1]),1)
        cd8_l.append(cd8_pred)
#         print(j/24,x_[j])
        time_l.append(j/24+10)
    return MAE(cd8_l,tcd8_target),cd8_l,time_l
def cal_time(x_,tumor):
    for i in range(len(x_.T[0])):
        if x_.T[0][i]+x_.T[1][i]>tumor:
            break
    return i

In [9]:
#initial r=0.1
Liu_all['time']=int(600)
mae,cd8_l,time_l=opt_fix_para()

100%|█████████████████████████████████████████| 120/120 [02:03<00:00,  1.03s/it]


In [10]:
mae

0.011121013692878202

In [21]:
time_l_=[]
for i in time_l:
    time_l_.append(int(i))

In [19]:
Liu_all['time']=time_l_

In [23]:
#fit r
for i in trange(len(Liu_all)):
    tcd8_target_s=tcd8_target[i]
    r_origin=r_l[i]
    ar_ratio=Liu_all['ar_ratio'][i]
    error_origin=np.abs(tcd8_target_s-cd8_l[i])
    if tcd8_target_s>cd8_l[i]:
        r=up_adjust(i,tcd8_target_s,r_origin,error_origin,ar_ratio)
    else:
        r=down_adjust(i,tcd8_target_s,r_origin,error_origin,ar_ratio)
    r_l[i]=r

100%|█████████████████████████████████████████| 120/120 [05:37<00:00,  2.81s/it]


In [24]:
#calculate mae after r fitting
mae,cd8_l,time_l=opt_fix_para()

100%|█████████████████████████████████████████| 120/120 [00:45<00:00,  2.64it/s]


In [25]:
#mae after r fitting
mae

0.001484007401008113

In [27]:
jc_scale=6

def down_adjust(i,cd8_target,r_origin,error_origin,ar_ratio):
    mu=Liu_all.loc[i,'mu']
    r_last=r_origin
    error_last=error_origin
    error=error_origin
    while error<=error_last:
        r=r_last-0.002
        jc=Liu_all.loc[i,'mhc_norm']
        jc_=(1-np.exp(-jc*jc_scale))/(1+np.exp(-jc*jc_scale))
        R_L_prop=Liu_all.loc[i,'Treg_pro']
        tumor=Liu_all.loc[i,'tumor_size']
        a=Liu_all.loc[i,'a']

        t1=Liu_all.loc[i,'time']
        x_=env.run(mu,r,jc_,R_L_prop,t1,a,ar_ratio)
        j=cal_time(x_,tumor)
    #     print('cd8+T:',x_[i][2])
        cd8_pred=min(x_[j][2]/(x_[j][0]+x_[j][1]),1)
        error_last=error
        error=np.abs(cd8_pred-cd8_target)
        r_last=r
    return r+0.002

def up_adjust(i,cd8_target,r_origin,error_origin,ar_ratio):
    mu=Liu_all.loc[i,'mu']
    r_last=r_origin
    error_last=error_origin
    error=error_origin
    while error<=error_last:
        r=r_last+0.002
        jc=Liu_all.loc[i,'mhc_norm']
        jc_=(1-np.exp(-jc*jc_scale))/(1+np.exp(-jc*jc_scale))
        R_L_prop=Liu_all.loc[i,'Treg_pro']
        tumor=Liu_all.loc[i,'tumor_size']
        a=Liu_all.loc[i,'a']
        
        t1=Liu_all.loc[i,'time']
        x_=env.run(mu,r,jc_,R_L_prop,t1,a,ar_ratio)
        j=cal_time(x_,tumor)
    #     print('cd8+T:',x_[i][2])
        cd8_pred=min(x_[j][2]/(x_[j][0]+x_[j][1]),1)
        error_last=error
        error=np.abs(cd8_pred-cd8_target)
        r_last=r
        if r>0.25:
            break
    return r-0.002   

In [30]:
# calculate error for jl fitting

In [31]:
def cal_jl_error(jl):

    cycle_l=[]
    count=0
    target=[]
    pred=[]
    for i in trange(120):
        target_re=Liu_all.loc[i,'BR']
        if  target_re=='MR' or target_re=='SD':
            pass
        else:
            mu=Liu_all.loc[i,'mu']
            r=r_l[i]

            jc=Liu_all.loc[i,'mhc_norm']
            jc_=(1-np.exp(-jc*jc_scale))/(1+np.exp(-jc*jc_scale))
            R_L_prop=Liu_all.loc[i,'Treg_pro']
            tumor=Liu_all.loc[i,'tumor_size']
            a=Liu_all.loc[i,'a']
            ar_ratio=Liu_all.loc[i,'ar_ratio']

            t1=Liu_all.loc[i,'time']
            x_=env.run(mu,r,jc_,R_L_prop,t1,a,ar_ratio)
            j=cal_time(x_,tumor)
            tumor_size_start=x_[j][0]+x_[j][1]
            cycles=Liu_all.loc[i,'cyclesOnTherapy']
            try:
                cycles=int(cycles)
            except:
                cycles=6

            x_begin=x_[j].tolist()
            x_begin.append(1)
            cd8_prop=Liu_all.loc[i,'tcd8_pop']
    #         print(cd8_prop,R_L_prop)
            x_begin[2]=cd8_prop*tumor_size_start
            for c in range(cycles):
                x_begin[-1]=1
                x_end=env.step(x_begin,mu,r,jc_,R_L_prop,jl)
                x_begin=x_end
    #             if x_end[0]+x_end[1]<1e2:
    #                 cycle_l.append(c)
    #                 break
    #             print(x_begin)
            tumor_size_end=x_end[0]+x_end[1]
            pred_re=1-tumor_size_end/tumor_size_start
            if pred_re>0:
                
                pred.append(1)
    #               
            else:
                pred.append(0)


            if target_re=='PR' or target_re=='CR':
                target.append(1)
            else:
                target.append(0)


    
    return target,pred

In [32]:
from sklearn.metrics import accuracy_score,recall_score

In [33]:
target,pred_2=cal_jl_error(0.11)
print(accuracy_score(target,pred_2))
print(recall_score(target,pred_2))

100%|█████████████████████████████████████████| 120/120 [01:21<00:00,  1.47it/s]

0.6310679611650486
0.6382978723404256


In [34]:
target,pred_1=cal_jl_error(0.1)
print(accuracy_score(target,pred_1))
print(recall_score(target,pred_1))

100%|█████████████████████████████████████████| 120/120 [01:20<00:00,  1.48it/s]

0.6407766990291263
0.574468085106383


In [35]:
target,pred_3=cal_jl_error(0.12)
print(accuracy_score(target,pred_3))
print(recall_score(target,pred_3))

100%|█████████████████████████████████████████| 120/120 [01:21<00:00,  1.48it/s]

0.6213592233009708
0.723404255319149


In [36]:
count={0:0,1:0,-1:0}
for i in np.array(target)-np.array(pred_3):
    count[i]+=1
count

{0: 64, 1: 13, -1: 26}

In [37]:
count={0:0,1:0,-1:0}
for i in np.array(target)-np.array(pred_1):
    count[i]+=1
count

{0: 66, 1: 20, -1: 17}

In [38]:
count={0:0,1:0,-1:0}
for i in np.array(target)-np.array(pred_2):
    count[i]+=1
count

{0: 65, 1: 17, -1: 21}